**Install and import necessary packages**

In [1]:
!pip install geemap
!pip install earthengine-api
!pip install PyCRS
!pip install restee
!pip install config
!pip install geojson


import geemap, ee, os, sys, pycrs, datetime
import geemap.colormaps as cm

from ipyleaflet import *
from ipywidgets import Label
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.dates as mdates
import matplotlib.ticker as ticker

import config
import json
import csv
import json
import geojson
import geopandas as gpd
import rasterio
import importlib
import numpy as np

import requests
import zipfile
import os
from os import listdir
from os.path import isfile,join

import restee as ree
from google.auth.transport.requests import AuthorizedSession

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 16.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 60.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 33.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   ------------------------ --------------- 11.5/19.1 MB 72.3 MB/s eta 0:00:01
   ---------------------------------------- 19.1/19.1 MB 50.1 MB/s eta 0:00:00
   -

**Import the Canopy Nitrogen Content models**

In [ ]:
import CNCmodel_Cab
import CNCmodel_Cprot

**Set your own Google Earth Engine credentials**

In [ ]:
yourcredentials = 'your-credentials'

!earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

class EESessionContainer(ree.EESession):
    def __init__(self, project, session):
        self._PROJECT = project
        self._SESSION = session

ee_session = EESessionContainer(yourcredentials, session)
ee.Initialize(ee_session.session.credentials, project=yourcredentials)

**User definition: choose region of interest, timeframe and asset path**

In [ ]:
# Now we define the region for retrieval which is a shapefile. You can upload your own shapefiles and retrieve data in that area too!
# Region of interest (MNI study site)
roi = ee.Geometry.Polygon(
    [[[11.67427222, 48.29512500],
      [11.67132778, 48.23868889],
      [11.73912778, 48.23709167],
      [11.74214722, 48.29352500]]])


# Start and end date and timesteps 
startDateStr = '2017-04-01'
endDateStr = '2017-09-20'
timeWindows = 7

# Your own Google Earth Engine asset path 
assetPath = 'projects/yourcredentials/assets/path/'

# Choose Cprot or Cab model
CNCmethod = 'Cprot'

In [ ]:
# Parameter settings
startDateGEE = ee.Date(startDateStr)
startDateGEE = ee.Date(startDateStr)
endDate = datetime.datetime.strptime(endDateStr, '%Y-%m-%d').date()

variables_GREEN = {'CNCmethod':['CNCmethod', 25, 0.001]}

S2MSI = ee.ImageCollection('COPERNICUS/S2_SR')

sys.setrecursionlimit(10000000)


# Cloud mask
def maskS2cloud(image):
  model = globals()['CNCmodel_' + CNCmethod]
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(model.scaleFactor_GREEN).copyProperties(qa).set('system:time_start', qa.get('system:time_start'))


# Desert and water masks
bare_cover = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('bare-coverfraction').lte(90);
lakes = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification').eq(80)
lakemask = lakes.eq(0);


# Some auxilary functions
def sequence_GREEN(CNCmethod):
	sequence_GREEN = []
	model = globals()['CNCmodel_' + CNCmethod]
	for i in range(0, model.XTrain_dim_GREEN):
		sequence_GREEN.append(str(i))
	return sequence_GREEN

def getInputDates(i):
  fecha_inicio = startDateGEE.advance(ee.Number(i).multiply(timeWindows),'day')
  fecha_fin = fecha_inicio.advance(timeWindows, 'day')
  fecha_str = datetime.datetime.utcfromtimestamp(fecha_inicio.getInfo()['value']/1000.0).strftime('%Y%m%d')
  return {'fecha_inicio':fecha_inicio, 'fecha_fin':fecha_fin, 'fecha_str':fecha_str}

def addVariables(image):
  date = ee.Date(image.get("system:time_start"));
  years = date.difference(ee.Date('1970-01-01'),'days');
  return image.addBands(ee.Image(years).rename('t').int());

**Implementation of Gaussian Processes Regression**

In [ ]:
def calculate_GREEN(fecha_inicio, fecha_fin, CNCmethod):

  model = globals()['CNCmodel_' + CNCmethod]
  image = ee.Image(S2MSI
  .filterDate(fecha_inicio, fecha_fin)
  .filterBounds(roi)
  .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 40)
  .map(maskS2cloud)
  .select(model.bands)
  .max()
  .clip(roi));

  im_norm_ell2D_hypell = image.subtract(model.mx_GREEN).divide(model.sx_GREEN).multiply(model.hyp_ell_GREEN).toArray().toArray(1);
  im_norm_ell2D = image.subtract(model.mx_GREEN).divide(model.sx_GREEN).toArray().toArray(1);
  PtTPt  = im_norm_ell2D_hypell.matrixTranspose().matrixMultiply(im_norm_ell2D).arrayProject([0]).multiply(-0.5);

  PtTDX  = ee.Image(model.X_train_GREEN).matrixMultiply(im_norm_ell2D_hypell).arrayProject([0]).arrayFlatten([sequence_GREEN(CNCmethod)]);
  arg1   = PtTPt.exp().multiply(model.hyp_sig_GREEN);
  k_star = PtTDX.subtract(model.XDX_pre_calc_GREEN.multiply(0.5)).exp().toArray();
  mean_pred = k_star.arrayDotProduct(model.alpha_coefficients_GREEN.toArray()).multiply(arg1);
  mean_pred = mean_pred.toArray(1).arrayProject([0]).arrayFlatten([[CNCmethod + '_CNC']]);
  mean_pred = mean_pred.add(model.mean_model_GREEN);

  k_star_uncert = PtTDX.subtract(model.XDX_pre_calc_GREEN.multiply(0.5)).exp().multiply(arg1).toArray();
  Vvector = ee.Image(model.Linv_pre_calc_GREEN).matrixMultiply(k_star_uncert.toArray(0).toArray(1)).arrayProject([0])
  Variance = ee.Image(model.hyp_sig_unc_GREEN).toArray().subtract(Vvector.arrayDotProduct(Vvector)).abs().sqrt()
  Variance = Variance.toArray(1).arrayProject([0]).arrayFlatten([[CNCmethod + '_UNCERTAINTY_CNC']])

  image= image.addBands(mean_pred);
  image = image.addBands(Variance);

  return image.select(CNCmethod + '_CNC', CNCmethod + '_UNCERTAINTY_CNC')

**Example Map for the MNI region (one image)**

In [ ]:
start_date = '2020-08-01'
end_date = '2020-08-03'

CNCimage = calculate_GREEN(start_date, end_date, 'Cprot')
image_veg_var = CNCimage.where(im1.lt(0), ee.Image(0.00001))

magma_palette = ['#fbfcbf', '#fceaac', '#fdd89a', '#fde1a3', '#fdcd90', '#fec488', '#feb179', '#fea671', '#fd9d6b', '#fc9366', '#f8755c', '#f56c5b', '#f2635c', '#ee5b5e', '#e75262', '#e14c66', '#da4769', '#d3426d', '#cb3e71', '#c23a75', '#b93778', '#b1357a', '#a9327c', '#9f2f7e', '#972c7f', '#8f2a80', '#872781', '#7f2481', '#762181', '#6e1e81', '#661a80', '#5e177f', '#5e177f', '#55137d', '#4d117a', '#450f76', '#3c0f71', '#341068', '#2a115c', '#221150', '#1b1044', '#150e38', '#0e0a2a', '#09071f', '#040415', '#01010b', '#000003'];
visParamsCNC = {'palette': magma_palette, 'min': 0.0, 'max': 25.0,'steps': 9}

Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map.addLayer(image_veg_var.select('Cprot_CNC'), visParamsCNC, 'CNC Prediction')
Map.centerObject(roi,13)
Map

**Function to iterate through all the images in the collection and export to your asset**

In [ ]:
def maploop():
    startDate = datetime.datetime.strptime(startDateStr, "%Y-%m-%d").date()
    daysIterations = abs((startDate - endDate) // timeWindows).days

    for i in range(0, daysIterations):
        print(getInputDates(i)['fecha_str'])
        imageHolder = ee.Image().set('system:time_start', startDateGEE.advance(ee.Number(i).multiply(timeWindows), 'days').millis())

        for variable_GREEN in variables_GREEN:
            params = variables_GREEN[variable_GREEN]
            variable = params[0]
            limitUp = params[1]
            limitDown = params[2]
            imagen = calculate_GREEN(getInputDates(i)['fecha_inicio'], getInputDates(i)['fecha_fin'], CNCmethod)
            imageHolder = imageHolder.addBands(imagen)

            image_export = imageHolder.select(CNCmethod + '_CNC', CNCmethod + 'UNCERTAINTY_CNC')
            image_export = image_export.mask(lakemask)
            image_export = image_export.mask(bare_cover)
            image_export = image_export.set('system:time_start', startDateGEE.advance(ee.Number(i).multiply(timeWindows),'days').millis());

            exportar = ee.batch.Export.image.toAsset(
            assetId=assetPath + getInputDates(i)['fecha_str'] + CNCmethod + '_CNC',
            image=image_export,
            maxPixels=4731453308,
            description=getInputDates(i)['fecha_str'] + CNCmethod + '_CNC',
            scale=20,
            region=roi
            )
            exportar.start()
            exportar.status()


**Execute export of time series CNC calculation**

In [ ]:
maploop()

20170401
20170402
20170403
20170404
20170405
20170406
20170407
20170408
20170409
20170410
20170411
20170412
20170413
20170414
20170415
20170416
20170417
20170418
20170419
20170420
20170421
20170422
20170423
20170424
20170425
20170426
20170427
20170428
20170429
20170430
20170501
20170502
20170503
20170504
20170505
20170506
20170507
20170508
20170509
20170510
20170511
20170512
20170513
20170514
20170515
20170516
20170517
20170518
20170519
20170520
20170521
20170522
20170523
20170524
20170525
20170526
20170527
20170528
20170529
20170530
20170531
20170601
20170602
20170603
20170604
20170605
20170606
20170607
20170608
20170609
20170610
20170611
20170612
20170613
20170614
20170615
20170616
20170617
20170618
20170619
20170620
20170621
20170622
20170623
20170624
20170625
20170626
20170627
20170628
20170629
20170630
20170701
20170702
20170703
20170704
20170705
20170706
20170707
20170708
20170709
20170710
20170711
20170712
20170713
20170714
20170715


20170716
20170717
20170718
20170719
20170720
20170721


20170722
20170723
20170724
20170725
20170726
20170727
20170728
20170729
20170730
20170731
20170801
20170802
20170803
20170804
20170805
20170806
20170807
20170808
20170809
20170810
20170811
20170812
20170813
20170814
20170815
20170816
20170817
20170818
20170819
20170820
20170821
20170822
20170823


20170824
20170825
20170826
20170827
20170828
20170829
20170830
20170831
20170901
20170902
20170903
20170904
20170905
20170906
20170907
20170908
20170909
20170910
20170911
20170912
20170913
20170914
20170915
20170916
20170917
20170918
20170919
